In [ ]:
import os
import sys
import git
import pathlib

import random

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

PROJ_ROOT_PATH = pathlib.Path(git.Repo('.', search_parent_directories=True).working_tree_dir)
PROJ_ROOT =  str(PROJ_ROOT_PATH)
if PROJ_ROOT not in sys.path:
    sys.path.append(PROJ_ROOT)

import libs.model_archs
import libs.utils
from libs.constants import MODELS_FOLDER

In [ ]:
# Limit GPU growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [ ]:
import libs.model_archs
import libs.utils
from libs.seeds import load_model_seeds
model_seeds = load_model_seeds()
model_seeds = [model_seeds[0]]

In [ ]:
# define dataset and model architecture
dataset = "cifar10"
model_arch = "vgg16F" # "fcA"

In [ ]:
# prepare data
dataset_loader = getattr(libs.utils, 'prepare_'+dataset)
(x_train, y_train), (x_test, y_test) = dataset_loader()

train_images_subset = x_train[0:1000] # out of 60000
train_labels_subset = y_train[0:1000]

In [ ]:
# set q_aware-training hyperparameters
# q_aware_batch_size = 500
# q_aware_n_epochs = 5

result = {}
for q_aware_batch_size in [128]:#, 256, 512, 1024]:
    for q_aware_n_epochs in [1]:#,2,5, 10, 20, 50]:
        for seed in model_seeds:
            # global seed
            # seed = model_seeds[0]
            tf.random.set_seed(seed)
            np.random.seed(seed)

            # load model
            model_type = dataset + "--" + model_arch
            model_instance = model_type + "-" + str(seed)
            model_filename = model_instance + ".h5"
            model_subdir = pathlib.Path(MODELS_FOLDER / model_arch)
            model_file = str(pathlib.Path(model_subdir/ model_filename))
            model = models.load_model(model_file)

            # create quantization aware model
            import tensorflow_model_optimization as tfmot

            # resulting model is quantization aware but not quantized (e.g. the weights are float32 instead of int8)
            quantize_model = tfmot.quantization.keras.quantize_model

            # q_aware stands for for quantization aware.
            q_aware_model = quantize_model(model)
            q_aware_model._name = "q_aware_"+model.name

            # `quantize_model` requires a recompile.
            q_aware_model.compile(optimizer='adam',
                                  loss=keras.losses.categorical_crossentropy,
                                  metrics=['accuracy'])

            # q_aware_model.summary()

            # finetune q_aware_model
            q_aware_model.fit(train_images_subset, 
                              train_labels_subset,
                              batch_size=q_aware_batch_size, 
                              epochs=q_aware_n_epochs, 
                              verbose=False,
                              validation_split=0.1)

            # # evaluate original model
            # _ , original_acc = model.evaluate(x_test, 
            #                        y_test, 
            #                        batch_size=32,
            #                       verbose=False)
            # result.setdefault(model_type,{}).setdefault(seed, {}).setdefault("acc", original_acc)

            # evaluate quantized model
            _, q_aware_acc = q_aware_model.evaluate(x_test, 
                                   y_test, 
                                   batch_size=32,
                                  verbose=False)

            q_aware_model_arch = "q_aware_" + model_arch
            q_aware_model_type = dataset + "--" + q_aware_model_arch
            result.setdefault(q_aware_model_type,{}).setdefault(q_aware_batch_size,{}).setdefault(q_aware_n_epochs,{}).setdefault(seed, {}).setdefault("acc", q_aware_acc)

In [ ]:
df = pd.DataFrame.from_dict({(i,j,k): result[i][j][k] # {1437: {'acc': 0.8871999979019165}, ...}
                             for i in result.keys() #['fashion--q_aware_lenetA]
                             for j in result[i].keys() #[32]
                             for k in result[i][j].keys() #[1,2,5,10,20,50]
                             },  
                             orient='index')

df = df.applymap(lambda x: x['acc'])

In [ ]:
df

In [ ]:
# # Create a dataframe from the dictionary
# # df = pd.DataFrame.from_dict(result)
# df = pd.DataFrame.from_dict({(i,j,k,l): result[i][j][k][l] 
#                            for i in result.keys() 
#                            for j in result[i].keys()
#                            for k in result[i][j].keys()
#                            for l in result[i][j][k].keys()},
#                        orient='index')
# """
# This code creates a dictionary comprehension that iterates over four levels of nested dictionaries (result[i][j][k][l]) and maps the values of the innermost level to a tuple of keys (i,j,k,l) using a dictionary key. The dictionary key is created by concatenating the tuple of keys with a comma, creating a tuple of tuples.

# The pd.DataFrame.from_dict() method is then called on this dictionary to create a DataFrame df. The orient parameter is set to 'index' to use the dictionary keys as the index of the DataFrame.

# Essentially, this code flattens the nested dictionary structure into a Pandas DataFrame where each row corresponds to a unique combination of the dictionary keys (i,j,k,l) and the corresponding value at the innermost level.
# """

# df.reset_index(inplace=True)
# df = df.pivot_table(index=['level_0', 'level_1', 'level_2'], columns='level_3', values='acc')
# df

In [ ]:
# # calculate max, avg, min
# df['max'] = df.max(axis=1)
# df['mean'] = df.mean(axis=1)
# df['min'] = df.min(axis=1)